In [2]:
from src.flight_analysis.scrape import Scrape

In [4]:
flights = Scrape("MUC", "LAX", "2023-10-28", export=False)
flights.run_scrape()

In [5]:
flights.data

,departure_datetime,arrival_datetime,airlines,travel_time,origin,destination,layover_n,layover_time,layover_location,price_eur,price_trend,price_value,access_date,one_way,has_train,days_advance
0,2023-10-28 11:30:00,2023-10-28 20:37:00,"[Delta, Virgin Atlantic, KLM]",1087,MUC,LAX,1,195,DTW,681,typical,None,2023-08-03 12:39:06.491987,True,False,85
1,2023-10-28 07:00:00,2023-10-28 11:50:00,[KLMDelta],830,MUC,LAX,1,75,AMS,768,typical,None,2023-08-03 12:39:06.492002,True,False,85
2,2023-10-28 07:15:00,2023-10-28 13:10:00,"[Air France, Delta]",895,MUC,LAX,1,100,CDG,770,typical,None,2023-08-03 12:39:06.492005,True,False,85
3,2023-10-28 10:25:00,2023-10-29 15:10:00,[Separate tickets booked together],2265,MUC,LAX,1,1450,LGW,489,typical,None,2023-08-03 12:39:06.492007,True,False,85
4,2023-10-28 14:50:00,2023-10-28 09:55:00,[KLMDelta],1010,MUC,LAX,1,255,AMS,684,typical,None,2023-08-03 12:39:06.492009,True,False,86
5,2023-10-28 18:10:00,2023-10-28 09:25:00,"[Vueling, Iberia]",1035,MUC,LAX,1,105,BCN,856,typical,None,2023-08-03 12:39:06.492011,True,False,86
6,2023-10-28 14:50:00,2023-10-28 11:25:00,[KLMDelta],865,MUC,LAX,1,110,AMS,883,typical,None,2023-08-03 12:39:06.492014,True,False,86
7,2023-10-28 18:30:00,2023-10-28 11:50:00,[Aer Lingus],965,MUC,LAX,1,125,DUB,959,typical,None,2023-08-03 12:39:06.492016,True,False,86


In [6]:
flights.url

'https://www.google.com/travel/flights?q=Flights%20to%20LAX%20from%20MUC%20on%202023-10-28%20oneway&curr=EUR&gl=IT'